In [17]:
!pip install youtube-transcript-api
!pip install pytube

In [18]:
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
import os
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

### 1. Simple Videos

In [20]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=g2bNHLeKlAg", add_video_info=True)
loader

In [22]:
result = loader.load()
result

[Document(page_content="foreign [Music] from redis he's a principal applied AI engineer and his key phrase to describe himself is curious which I find very interesting especially given that if you ask hiring managers for the most desirable features and attributes of a specific type of data scientist usually what you'll hear are some of the softer skills things like ability to communicate or understand business problems and curiosity is one of the top named items that hiring managers are looking for so welcome Sam to the stage [Applause] thank you hey everybody hopefully everybody's still awake all right my name is Sam partie thanks for the introduction I'm here to talk about AI powered search and Vector databases so first I'll ask a question who's had an argument in the last month ended by just Google it yeah come on more of you definitely more of you it happens all the time search is ingrained in our lives almost every day you search something so what I'm talking about today is going 

In [23]:
type(result)

list

In [24]:
print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")

Found video from Southern Data Science Conference that is 975 seconds long


In [40]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [26]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

" Sam Partie, a Principal Applied AI Engineer, discussed AI-powered search and vector databases. He explained how vector embeddings can be used to calculate the similarity between pieces of unstructured data, and how vector databases can be used to store and serve this data. He then demonstrated two applications of vector search: a fashion product finder and an archive search for medical diagnosis papers. Lastly, he discussed Relevance AI, a platform built on top of Redis' vector search capability."

### 2. Long Videos

In [27]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=6Ub7Z1AGIuk", add_video_info=True)
loader

In [29]:
result = loader.load()
result

[Document(page_content="somehow five years later covet hits and you know what they decide to do they're like you know what let's go all in on creating like safety safety stuff for covert like masks respirators um gloves things like that and they become one of the like core providers right at the start of covet they shifted all their production manufacturing stuff they're like forget the vodka bottles forget the LED screens we need n95 masks and we need respirators and we need you know these testing kits and we need and they started producing this stuff and they started just winning all these contracts [Music] we're live Sean uh we have a bunch of stuff you you have a you have a a full menu over here and so do I yeah all right I want you to look at my menu here and I want you to just pick something off the menu go ahead and I'll Riff Off it we don't prefer people who they don't they don't know we only write like one or two words so you get like a teaser you're like man what the hell is 

In [30]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 14199 tokens (13943 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

Problem, your transcript/document is too long

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [34]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
text_splitter

In [35]:
texts = text_splitter.split_documents(result)
texts

[Document(page_content="somehow five years later covet hits and you know what they decide to do they're like you know what let's go all in on creating like safety safety stuff for covert like masks respirators um gloves things like that and they become one of the like core providers right at the start of covet they shifted all their production manufacturing stuff they're like forget the vodka bottles forget the LED screens we need n95 masks and we need respirators and we need you know these testing kits and we need and they started producing this stuff and they started just winning all these contracts [Music] we're live Sean uh we have a bunch of stuff you you have a you have a a full menu over here and so do I yeah all right I want you to look at my menu here and I want you to just pick something off the menu go ahead and I'll Riff Off it we don't prefer people who they don't they don't know we only write like one or two words so you get like a teaser you're like man what the hell is 

In [36]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts[:4])



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Write a concise summary of the following:


"somehow five years later covet hits and you know what they decide to do they're like you know what let's go all in on creating like safety safety stuff for covert like masks respirators um gloves things like that and they become one of the like core providers right at the start of covet they shifted all their production manufacturing stuff they're like forget the vodka bottles forget the LED screens we need n95 masks and we need respirators and we need you know these testing kits and we need and they started producing this stuff and they started just winning all these contracts [Music] we're live Sean uh we have a bunch of stuff you you have a you have a a full menu over here and so do I yeah all right I want you to look at my menu here and I want you to just pick something off the menu go ahead and I'll Riff Off it we don't prefer people who they don't they don't kno



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"

Covet shifted their production from vodka bottles and LED screens to safety equipment such as masks, respirators, and gloves when the pandemic hit. They won contracts and became one of the core providers. The speaker then talks about a rich neighbor who was balling out of control, with a snow machine and a Christmas tree-lit house.

 A company created a product in 2014 that allowed people to program messages onto vodka bottles with LED signs. This product was successful enough to attract Shaq as an advisor. When the pandemic hit, the company decided to use their resources to create a snow hill in a driveway, complete with a fake Santa, for a Thursday afternoon.

 A local East Bay company shifted their production from vodka bottles and LED screens to safety equipment such as masks, respirators, and gloves at the start of the COVID-19 pa

" A company shifted their production from vodka bottles and LED screens to safety equipment such as masks, respirators, and gloves at the start of the COVID-19 pandemic. They have since won multiple contracts and are now the sole testing provider for places such as the Lakers Stadium and Levi's Stadium. Another person in a Mastermind group was discussing a business venture involving mobile Covid testing trucks, but it was unclear how it ended."

### 3. Multiple Videos

In [38]:
youtube_url_list = ["https://www.youtube.com/watch?v=AXq0QHUwmh8", "https://www.youtube.com/watch?v=EwHrjZxAT7g"]

texts = []

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

for url in youtube_url_list:
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    result = loader.load()
    
    texts.extend(text_splitter.split_documents(result))

In [39]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)

' This article covers the process of building a backyard golf green, including design, materials, site work, and ongoing maintenance. It also discusses the cost of the project, tips on how to design the site layout, and what to consider when buying land. Bella Ramsey and Pedro Pascal also discuss their experiences on the set of Game of Thrones and The Last of Us, including their first impressions of each other, what they took from set, and their first jobs.'